In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


In [2]:
x_frame1_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame1_1.25_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame2_1.25_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame3_1.25_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame_noise_1.25_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "dual_natural", weight='tan')
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

658


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
x_adj=np.array(x_adj)

In [11]:
def calculate_g(data):
    temp2=[]
    for vs_out in data:
        a = np.sum(vs_out, axis=0).reshape(-1, 1)
        b = np.sum(vs_out != 0, axis=1).reshape(-1, 1)
        temp2.append(np.concatenate((a, b), axis=1))
        
    return temp2

In [12]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
658


In [13]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(calculate_g, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 1.536695957183838 seconds ---


In [14]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [15]:
x_g=np.array(x_g)


In [16]:
print(x_g.shape)

(31578, 56, 2)


In [17]:
x_g[0]

array([[ 0.09838868,  1.        ],
       [ 0.11450197,  2.        ],
       [ 0.07451176,  4.        ],
       [ 0.11035158,  2.        ],
       [ 0.13109312,  7.        ],
       [ 0.15080921,  6.        ],
       [ 0.12935389,  5.        ],
       [ 0.14367679,  3.        ],
       [ 0.12677007,  5.        ],
       [ 0.10105801,  7.        ],
       [ 0.0827637 ,  3.        ],
       [ 0.0870013 ,  4.        ],
       [ 0.26430969, 14.        ],
       [ 0.09950301,  6.        ],
       [ 0.08715825,  5.        ],
       [ 0.07617193,  5.        ],
       [ 0.09493006,  5.        ],
       [ 0.20422802, 12.        ],
       [ 0.10791018,  2.        ],
       [ 0.09372972,  7.        ],
       [ 0.08817553,  7.        ],
       [ 0.04995122,  5.        ],
       [ 0.0499268 ,  4.        ],
       [ 0.06317146,  7.        ],
       [ 0.06809787,  7.        ],
       [ 0.10257979,  4.        ],
       [ 0.3138469 , 19.        ],
       [ 0.1254883 ,  2.        ],
       [ 0.12046637,

In [18]:
x_g_flat=np.array([np.array(i.T.flatten()).ravel() for i in x_g])


In [19]:
x_g_flat[0]

array([ 0.09838868,  0.11450197,  0.07451176,  0.11035158,  0.13109312,
        0.15080921,  0.12935389,  0.14367679,  0.12677007,  0.10105801,
        0.0827637 ,  0.0870013 ,  0.26430969,  0.09950301,  0.08715825,
        0.07617193,  0.09493006,  0.20422802,  0.10791018,  0.09372972,
        0.08817553,  0.04995122,  0.0499268 ,  0.06317146,  0.06809787,
        0.10257979,  0.3138469 ,  0.1254883 ,  0.12046637,  0.0387116 ,
        0.10339004,  0.0942383 ,  0.06958011,  0.15655817,  0.16375737,
        0.14444991,  0.17590337,  0.17297368,  0.31122403,  0.13431808,
        0.08544925,  0.13407393,  0.17243254,  0.03649622,  0.04361645,
        0.07914798,  0.06982424,  0.4746936 ,  0.13995366,  0.10506189,
        0.12373866,  0.12609866,  0.14087329,  0.04777022,  0.05510607,
        0.07668055,  1.        ,  2.        ,  4.        ,  2.        ,
        7.        ,  6.        ,  5.        ,  3.        ,  5.        ,
        7.        ,  3.        ,  4.        , 14.        ,  6.  

In [20]:
x_train, x_temp, y_train, y_temp = train_test_split(x_g_flat, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [21]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(22104, 112)
(4737, 112)
(4737, 112)


In [22]:
import tensorflow as tf


2023-08-13 10:18:05.576866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 10:18:05.745579: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 10:18:05.749524: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-

In [23]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 10:18:17.867601: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 10:18:17.867636: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 10:18:17.867655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0155): /proc/driver/nvidia/version does not exist
2023-08-13 10:18:17.868325: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow

In [24]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 112, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 56, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 56, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 28, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dense (Dense)               (None, 100)               8

In [25]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F8.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [27]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
691/691 [==============================] - 4s 4ms/step - loss: 0.2282 - acc: 0.9120 - val_loss: 0.1097 - val_acc: 0.9609
Epoch 2/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0862 - acc: 0.9705 - val_loss: 0.0804 - val_acc: 0.9759
Epoch 3/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0726 - acc: 0.9758 - val_loss: 0.0774 - val_acc: 0.9772
Epoch 4/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0707 - acc: 0.9759 - val_loss: 0.0969 - val_acc: 0.9681
Epoch 5/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0618 - acc: 0.9791 - val_loss: 0.0866 - val_acc: 0.9721
Epoch 6/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0613 - acc: 0.9798 - val_loss: 0.0837 - val_acc: 0.9747
Epoch 7/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0604 - acc: 0.9797 - val_loss: 0.0710 - val_acc: 0.9783
Epoch 8/30
691/691 [==============================] - 3s 4ms/step - loss: 0.

Epoch 29/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0191 - acc: 0.9930 - val_loss: 0.1272 - val_acc: 0.9776
Epoch 30/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0144 - acc: 0.9951 - val_loss: 0.1331 - val_acc: 0.9816


In [28]:
model3.save_weights('/home/nvekariy/signal_code/New_neural_code_SNR_1.25_h/F8.h5')


In [29]:
model3.load_weights('/home/nvekariy/signal_code/New_neural_code_SNR_1.25_h/F8.h5')


In [30]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

149/149 [==============================] - 0s 2ms/step


In [31]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[ 940    1    0    6]
 [   0  948   30    4]
 [   1   29  969    2]
 [   8    5    1 1793]]


In [32]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [33]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

98.16339455351488
97.94410479045699
98.16339455351488
98.16361148881302
